<a href="https://colab.research.google.com/github/HussamCheema/DermCNN/blob/master/DermCNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PART 1

---



In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras import optimizers


Using TensorFlow backend.


In [2]:
!git clone https://github.com/HussamCheema/DermCNN.git

Cloning into 'DermCNN'...
remote: Enumerating objects: 11348, done.
remote: Counting objects: 100% (11348/11348), done.
remote: Compressing objects: 100% (11348/11348), done.
remote: Total 11348 (delta 6), reused 11335 (delta 0), pack-reused 0
Receiving objects: 100% (11348/11348), 438.26 MiB | 13.93 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Checking out files: 100% (11360/11360), done.


# Data Pre-Processing

In [0]:
data = []
labels = []

In [0]:
batch_size = 100
epochs = 15

In [0]:
for folder in range(1,8):
  for image in os.listdir(os.getcwd()+'/DermCNN/'+str(folder)+'/'):
    try:
      x = cv2.imread(os.getcwd()+'/DermCNN/'+str(folder)+'/'+image, cv2.IMREAD_GRAYSCALE)
      x = cv2.resize(x, (150,150))
      data.append(x)
      labels.append(folder-1)
    except: pass


In [16]:
data = np.array(data)
data = data.reshape((data.shape)[0],(data.shape)[1],(data.shape)[2],1)
labels = np.array(labels)
print(len(data))
print(len(labels))
print(data.shape)
print(labels.shape)

11355
11355
(11355, 150, 150, 1)
(11355,)


In [17]:
data = data.astype('float')
data /= 255
labels = keras.utils.to_categorical(labels, 7)
print(labels.shape)

(11355, 7)


#Shuffle Data


In [0]:
def shuffle(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [0]:
for i in range(10):
    shuffle(data,labels)

# CNN MODEL

In [0]:
model = Sequential()
model.add(Conv2D(kernel_size=(3,3),filters=64,input_shape=(150,150,1),activation="relu",padding="valid"))
model.add(Conv2D(kernel_size=(3,3),filters=64,activation="relu",padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(kernel_size=(3,3),filters=32,activation="relu",padding="same"))
model.add(Conv2D(kernel_size=(2,2),filters=22,activation="relu",padding="same"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(Flatten())
model.add(Dropout(0.7))
model.add(Dense(128,activation="relu"))
model.add(Dense(7,activation="softmax"))

model.summary()
model.compile(optimizer = keras.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])


X = model.fit(data,labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_split=0.3)

plt.plot(X.history['loss'])
plt.plot(X.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 64)      640       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 148, 148, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 74, 74, 32)        18464     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 74, 74, 22)        2838      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 22)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 28512)             0         
__________